In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

### **Data Reading**

In [0]:
df = spark.read.format("parquet") \
          .load("abfss://bronze@databricksend2end.dfs.core.windows.net/products")

display(df)

In [0]:
df = df.drop("_rescued_data")

In [0]:
display(df)

In [0]:
df.createOrReplaceTempView("vw_products")

### **Functions**

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_cata.bronze.discount_func(p_price double)
RETURNS DOUBLE
LANGUAGE SQL
RETURN p_price * 0.90

In [0]:
%sql
select product_id, price, databricks_cata.bronze.discount_func(price) as discounted_price
from vw_products;

In [0]:
df = df.withColumn("discounted_price", expr("databricks_cata.bronze.discount_func(price)"))
display(df)

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_cata.bronze.upper_func(p_brand STRING)
RETURNS string
LANGUAGE PYTHON
AS
$$
  return p_brand.upper()
$$

In [0]:
%sql
SELECT product_id
      , brand
      , databricks_cata.bronze.upper_func(brand) as brand_upper
FROM vw_products

### **Data Writing**

In [0]:
df.write.mode("overwrite") \
  .format("delta") \
  .option("path","abfss://silver@databricksend2end.dfs.core.windows.net/products") \
  .save()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_cata.silver.products
USING DELTA
LOCATION 'abfss://silver@databricksend2end.dfs.core.windows.net/products'

In [0]:
%sql
SELECT * FROM databricks_cata.silver.products